In [ ]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
from math import ceil
import os
from typing import Optional
from pathlib import Path
from datetime import datetime

from pypsa.statistics import (
    get_bus_and_carrier,
    get_carrier_and_bus_carrier,
    get_bus_and_carrier_and_bus_carrier,
    get_name_bus_and_carrier,
)

In [ ]:
n = pypsa.Network("elec_s_20_ec_lv1.0_500SEG_E-G.nc")

In [ ]:
trn = n.loads_t.p_set[[x for x in n.loads_t.p_set if "trn-elec" in x]]
trn = trn.rename(columns={x: x.split("trn-")[1] for x in trn.columns})
trn = trn.T.groupby(level=0).sum().T
trn.sum()
# n.loads_t.p_set

In [ ]:
bvmt = 3215  # bmt
perct = 9.309

bvmt * perct / 100

In [ ]:
def get_profile(df: pd.DataFrame, bus: Optional[str] = None) -> pd.DataFrame:

    if not bus:
        bus = df.columns[0]

    profile = df[bus].to_frame()
    profile = (profile / profile.sum()) * 100

    profile.index = pd.DatetimeIndex(profile.index)
    profile["hour"] = profile.index.map(lambda x: x.hour)
    profile["day"] = profile.index.map(lambda x: x.timetuple().tm_yday - 1)
    profile = profile.reset_index(drop=True)

    return profile.pivot(index="hour", columns="day", values=bus)

In [ ]:
def plot_ev_load_per_day(
    root_dir: str,
    bus: Optional[str] = None,
    sharey: bool = False,
    log: bool = False,
) -> tuple:
    """
    Plots pre-aggregated ev load profiles
    """

    vehicles = ("light-duty", "med-duty", "heavy-duty", "bus")

    data = {
        v: pd.read_csv(Path(root_dir, f"transport_{v}_electricity.csv"), index_col=0)
        for v in vehicles
    }

    nrows = ceil(len(vehicles) / 2)

    fig, axs = plt.subplots(
        ncols=2,
        nrows=nrows,
        figsize=(14, 5 * nrows),
        sharey=sharey,
    )

    ylabel = "Perecent of Yearly Load (%)"

    row = 0
    col = 0

    for i, vehicle in enumerate(vehicles):
        row = i // 2
        col = i % 2

        vehicle_data = data[vehicle]
        if vehicle_data.empty:
            continue

        df = get_profile(vehicle_data)
        avg = df.mean(axis=1)

        palette = sns.color_palette(["lightgray"], df.shape[1])

        if nrows > 1:

            sns.lineplot(
                df,
                color="lightgray",
                legend=False,
                palette=palette,
                ax=axs[row, col],
                linewidth=0.25,
            )
            sns.lineplot(avg, ax=axs[row, col], linewidth=3)

            axs[row, col].set_xlabel("")
            axs[row, col].set_ylabel(ylabel)
            axs[row, col].set_title(f"{vehicle} load")

            if log:
                axs[row, col].set(yscale="log")

        else:

            sns.lineplot(
                df,
                color="lightgray",
                legend=False,
                palette=palette,
                ax=axs[i],
                linewidth=0.25,
            )
            sns.lineplot(avg, ax=axs[i], linewidth=3)

            axs[i].set_xlabel("")
            axs[i].set_ylabel(ylabel)
            axs[i].set_title(f"{vehicle} load")

            if log:
                axs[i].set(yscale="log")

    fig.tight_layout()

    return fig, axs

In [ ]:
plot_ev_load_per_day("./../../../../resources/Default/texas/")

In [ ]:
def plot_service_load_per_day(
    root_dir: str,
    sector: str,
    bus: Optional[str] = None,
    sharey: bool = False,
    log: bool = False,
) -> tuple:
    """
    Plots pre-aggregated ev load profiles
    """

    assert sector in ("residential", "commercial")

    fuels = ("cooling", "electricity", "heating")

    data = {
        fuel: pd.read_csv(Path(root_dir, f"{sector}_{fuel}.csv"), index_col=0)
        for fuel in fuels
    }

    nrows = ceil(len(fuels) / 2)

    fig, axs = plt.subplots(
        ncols=2,
        nrows=nrows,
        figsize=(14, 5 * nrows),
        sharey=sharey,
    )

    ylabel = "Perecent of Yearly Load (%)"

    row = 0
    col = 0

    for i, fuel in enumerate(fuels):
        row = i // 2
        col = i % 2

        service_data = data[fuel]
        if service_data.empty:
            continue

        df = get_profile(service_data)
        avg = df.mean(axis=1)

        palette = sns.color_palette(["lightgray"], df.shape[1])

        if nrows > 1:

            sns.lineplot(
                df,
                color="lightgray",
                legend=False,
                palette=palette,
                ax=axs[row, col],
                linewidth=0.25,
            )
            sns.lineplot(avg, ax=axs[row, col], linewidth=3)

            axs[row, col].set_xlabel("")
            axs[row, col].set_ylabel(ylabel)
            axs[row, col].set_title(f"{sector} {fuel} load")

            if log:
                axs[row, col].set(yscale="log")

        else:

            sns.lineplot(
                df,
                color="lightgray",
                legend=False,
                palette=palette,
                ax=axs[i],
                linewidth=0.25,
            )
            sns.lineplot(avg, ax=axs[i], linewidth=3)

            axs[i].set_xlabel("")
            axs[i].set_ylabel(ylabel)
            axs[i].set_title(f"{sector} {fuel} load")

            if log:
                axs[i].set(yscale="log")

    fig.tight_layout()

    return fig, axs

In [ ]:
plot_service_load_per_day("./../../../../resources/Default/texas/", "residential")

In [ ]:
plot_service_load_per_day("./../../../../resources/Default/texas/", "commercial")

In [ ]:
def plot_industrial_load_per_day(
    root_dir: str,
    bus: Optional[str] = None,
    sharey: bool = False,
    log: bool = False,
) -> tuple:
    """
    Plots pre-aggregated ev load profiles
    """

    fuels = ("electricity", "heating")

    data = {
        fuel: pd.read_csv(Path(root_dir, f"industry_{fuel}.csv"), index_col=0)
        for fuel in fuels
    }

    nrows = ceil(len(fuels) / 2)

    fig, axs = plt.subplots(
        ncols=2,
        nrows=nrows,
        figsize=(14, 5 * nrows),
        sharey=sharey,
    )

    ylabel = "Perecent of Yearly Load (%)"

    row = 0
    col = 0

    for i, fuel in enumerate(fuels):
        row = i // 2
        col = i % 2

        service_data = data[fuel]
        if service_data.empty:
            continue

        df = get_profile(service_data)
        avg = df.mean(axis=1)

        palette = sns.color_palette(["lightgray"], df.shape[1])

        if nrows > 1:

            sns.lineplot(
                df,
                color="lightgray",
                legend=False,
                palette=palette,
                ax=axs[row, col],
                linewidth=0.25,
            )
            sns.lineplot(avg, ax=axs[row, col], linewidth=3)

            axs[row, col].set_xlabel("")
            axs[row, col].set_ylabel(ylabel)
            axs[row, col].set_title(f"industrial {fuel} load")

            if log:
                axs[row, col].set(yscale="log")

        else:

            sns.lineplot(
                df,
                color="lightgray",
                legend=False,
                palette=palette,
                ax=axs[i],
                linewidth=0.25,
            )
            sns.lineplot(avg, ax=axs[i], linewidth=3)

            axs[i].set_xlabel("")
            axs[i].set_ylabel(ylabel)
            axs[i].set_title(f"industrial {fuel} load")

            if log:
                axs[i].set(yscale="log")

    fig.tight_layout()

    return fig, axs

In [ ]:
plot_industrial_load_per_day("./../../../../resources/Default/texas/")